In [1]:
import numpy as np
from scipy.special import expit

In [2]:
def irt1pl(theta, b):
    return expit(theta -b)

In [3]:
def irt2pl(theta, a, b):
    return expit(a * (theta - b))

In [4]:
def irt3pl(theta, a, b, c):
    return c + (1 - c) * expit(a * (theta - b))

In [5]:
from scipy.stats import chi2, norm, beta

In [6]:
def simulate_irt_responses(model="1PL", num_items=50, theta=0.5, threshold=0.5, seed=None):

    if seed is not None:
        np.random.seed(seed)

    # Sinh tham số
    a_raw = chi2.rvs(df=3, size=num_items)
    a = np.interp(a_raw, (a_raw.min(), a_raw.max()), (0.1, 2.8))
    b = np.clip(norm.rvs(loc=0, scale=1, size=num_items), -3, 3)
    c_raw = beta.rvs(5, 17, size=num_items)
    c = c_raw * 0.35

    # Điều chỉnh theo mô hình
    if model == "1PL":
        a = np.ones(num_items)
        c = np.zeros(num_items)
    elif model == "2PL":
        c = np.zeros(num_items)
    elif model != "3PL":
        raise ValueError("Chỉ hỗ trợ '1PL', '2PL', hoặc '3PL'.")

    # Tính xác suất đúng
    z = a * (theta - b)
    p = c + (1 - c) * 1 / (1 + np.exp(-z))

    # Phản hồi xác định: p >= threshold → đúng (1), ngược lại sai (0)
    responses = (p >= threshold).astype(int)

    return responses, {"a": a, "b": b, "c": c, "theta": theta, "p": p}

In [7]:
theta_true = 0.5
responses1pl, dl1pl = simulate_irt_responses(theta = theta_true)

In [8]:
responses1pl

array([0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 0])

In [9]:
b_true = dl1pl['b']
b_true

array([ 2.17572595, -1.57445191,  0.30132349,  0.07463089,  0.21715557,
        1.85270085, -0.7090541 ,  1.73719313,  1.27032561, -0.28398035,
       -0.38621224,  0.51238161, -0.9437526 , -0.67289548,  0.79590398,
        0.01180536,  0.23041895,  1.41098731, -0.25749424,  0.42400523,
       -0.02982384,  0.74808787, -0.15166498, -0.65929085,  0.18094918,
       -0.34425036,  0.35008138, -0.73564744,  0.29432311, -0.52837686,
       -0.27815073,  0.98021986,  0.23781655,  1.5601095 ,  3.        ,
       -0.78782472, -1.33730072,  0.85518083, -0.88960827,  1.91858278,
        2.14175904, -1.05046825,  0.10824696,  0.77370543,  0.3579491 ,
        0.18460627, -1.05308317, -0.584593  , -0.29476064,  0.86038822])

In [10]:
def uoc_luong_nlts_1pl(theta_init, b, u, max_iterations = 100, tol = 10e-6):
    theta_es = theta_init
    for i in range(max_iterations):
        P = irt1pl(theta_es, b)
        Q = 1 - P
        L1 = 0
        L2 = 0
        for j in range(len(b)):
            L1 += u[j] - P[j]
            L2 += P[j] * Q[j]
        delta = L1 / L2
        if np.abs(delta) < tol:
            print(f"Hội tụ sau {i + 1} lần lặp !!")
            break
        theta_es = theta_es  + delta
    SE = 1 / np.sqrt(L2)
    return theta_es, SE

In [11]:
theta_init = 0
theta_es_1pl, se_1pl = uoc_luong_nlts_1pl(theta_init, b_true, responses1pl)

Hội tụ sau 4 lần lặp !!


In [12]:
print(theta_es_1pl, se_1pl)

1.119882085016463 0.3344991862189013


In [13]:
theta_true = 0.5
responses2pl, dl2pl = simulate_irt_responses(model = "2PL", theta = theta_true)

In [14]:
responses2pl

array([1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0])

In [15]:
a_true_2pl = dl2pl['a']
a_true_2pl

array([0.53809016, 0.30604543, 1.52137972, 2.21716063, 0.4634116 ,
       0.39703571, 1.93456359, 2.8       , 0.52125568, 1.24656107,
       1.89720859, 0.1       , 1.14055243, 0.18178982, 0.97151253,
       0.8816913 , 0.82431513, 0.68036056, 0.38637447, 0.29636385,
       1.10351841, 2.31837308, 1.01718014, 0.49528282, 0.59038009,
       0.38267345, 0.53942554, 0.35531566, 1.00355245, 2.65181023,
       0.38146753, 2.36480314, 0.84585521, 1.69154644, 1.13701455,
       1.05961032, 0.87830164, 0.79791202, 0.70907015, 1.29377297,
       1.10426686, 0.86627138, 0.37379408, 0.97540582, 0.95780489,
       0.45232268, 1.46769941, 0.24256954, 1.18765596, 0.88270473])

In [16]:
b_true_2pl = dl2pl['b']
b_true_2pl

array([-0.69806194, -0.05716732, -1.35588873, -0.77965187,  0.43561013,
        0.79985254,  0.91059549, -0.58586674,  0.74375645, -0.86818832,
       -0.61654818, -0.41397429,  0.59028111,  2.06242523, -0.83863285,
        0.78184245,  0.31482925,  2.33672276,  0.87667223,  1.13529795,
       -1.31880766,  1.23364224, -0.55132638,  1.31188891, -0.69584254,
       -0.31995425, -0.01801304, -1.7425206 ,  1.05237881,  0.61280193,
        0.41250786,  0.8865471 ,  1.45440785, -0.14628361,  1.58149816,
        1.06065166, -1.60701027, -1.13307819,  1.24676837, -0.53733867,
       -0.65281202, -1.95718453, -0.71841359, -1.53817905, -0.2792196 ,
       -0.54902236,  0.82216117,  0.3618057 ,  0.69957956,  0.51041689])

In [17]:
def uoc_luong_nlts_2pl(theta_init, a, b, u, max_iterations = 100, tol = 10e-6):
    theta_es = theta_init
    for i in range(max_iterations):
        P = irt2pl(theta_es, a, b)
        Q = 1 - P
        L1 = 0
        L2 = 0
        for j in range(len(b)):
            L1 += a[j] * (u[j] - P[j])
            L2 += a[j] ** 2 * P[j] * Q[j]
        delta = L1 / L2
        if np.abs(delta) < tol:
            print(f"Hội tụ sau {i + 1} lần lặp !!")
            break
        theta_es = theta_es  + delta
    SE = 1 / np.sqrt(L2)
    return theta_es, SE

In [18]:
theta_es_2pl, se_2pl = uoc_luong_nlts_2pl(theta_init, a_true_2pl, b_true_2pl, responses2pl)

Hội tụ sau 3 lần lặp !!


In [19]:
print(theta_es_2pl, se_2pl)

0.27221008995228824 0.3010024752207827


In [20]:
theta_true = 0.5
responses3pl, dl3pl = simulate_irt_responses(model = "3PL", theta = theta_true)

In [21]:
a_true_3pl = dl3pl['a']
a_true_3pl

array([0.19385792, 0.9336937 , 0.5806382 , 0.51834546, 1.63106975,
       0.29382687, 0.98256056, 0.62777209, 0.87210075, 0.90715701,
       1.04284296, 0.43194615, 1.09223293, 0.85084007, 2.02565968,
       0.67377007, 0.22742249, 0.46483375, 0.69435378, 1.138575  ,
       1.38263046, 0.42006504, 0.26593109, 1.07028273, 0.60856866,
       0.3345301 , 0.31319795, 0.92599268, 0.61180632, 0.58641595,
       0.37101603, 1.09472147, 0.71301711, 2.01828014, 0.17640163,
       2.23066965, 0.61716184, 0.28442307, 0.86011225, 0.31851788,
       0.11980599, 0.80689227, 0.59866082, 1.79293385, 0.1       ,
       0.82358816, 1.01592374, 0.37077247, 0.40190871, 2.8       ])

In [22]:
b_true_3pl = dl3pl['b']
b_true_3pl

array([ 0.75742194, -0.82913422, -0.11689556,  2.05549533, -0.75685009,
        2.15216971,  0.82647292,  0.98174203,  0.8605978 , -1.07498207,
       -0.64498452,  1.34842109,  0.20537701, -0.83092521,  0.31542367,
        0.11846288,  0.65631521,  0.10366101,  0.44209237, -1.79704229,
       -0.73538199,  0.59230566,  0.76334184, -0.59699429, -0.90315399,
        1.12988963, -0.2712599 , -1.19165079,  1.12347177,  1.04121065,
        0.18887421,  0.9428857 , -0.05642503, -0.3553928 ,  0.0162596 ,
       -0.19930797, -0.55027073, -1.49149677,  0.8998721 , -1.42195325,
        0.29222698, -1.28102435, -0.43161193,  3.        ,  1.34624029,
       -0.8112498 , -0.32045106, -1.06292257,  1.85908119,  0.14813721])

In [23]:
c_true_3pl = dl3pl['c']
c_true_3pl

array([0.07076895, 0.10967699, 0.06266502, 0.11782579, 0.12688836,
       0.0528175 , 0.0838145 , 0.09040381, 0.09503477, 0.11348364,
       0.08027018, 0.08671374, 0.05285771, 0.07353311, 0.07807632,
       0.06512492, 0.09303826, 0.02935055, 0.13083939, 0.14100846,
       0.05574347, 0.05698147, 0.0995011 , 0.02479306, 0.14972351,
       0.10921562, 0.10196496, 0.05778576, 0.05205704, 0.05516753,
       0.06363843, 0.15967341, 0.10207041, 0.13479671, 0.09276371,
       0.07308593, 0.05286442, 0.04677227, 0.0296378 , 0.02752491,
       0.12878285, 0.10062505, 0.07728754, 0.03521767, 0.0866674 ,
       0.08922617, 0.10510139, 0.04125483, 0.09768616, 0.09753351])

In [24]:
def uoc_luong_nlts_3pl(theta_init, a, b, c, u, max_iterations = 100, tol = 10e-6):
    theta_es = theta_init
    for i in range(max_iterations):
        P = irt3pl(theta_es, a, b, c)
        Q = 1 - P
        L1 = 0
        L2 = 0
        for j in range(len(b)):
            L1 += a[j] * (P[j] - c[j]) * (u[j] - P[j]) / ((1 - c[j]) * P[j]) 
            L2 += a[j] ** 2 * (Q[j] * ((P[j] - c[j]) ** 2)) / (P[j] * ((1 - c[j]) ** 2))
        delta = L1 / L2
        if np.abs(delta) < tol:
            print(f"Hội tụ sau {i + 1} lần lặp !!")
            break
        theta_es = theta_es  + delta
    SE = 1 / np.sqrt(L2)
    return theta_es, SE

In [25]:
theta_es_3pl, se_3pl = uoc_luong_nlts_3pl(theta_init, a_true_3pl, b_true_3pl, c_true_3pl, responses3pl)

Hội tụ sau 6 lần lặp !!


In [26]:
print(theta_es_3pl, se_3pl)

1.3443809891431808 0.5032532416007595


In [27]:
def uoc_luong_nlts_map_1pl(theta_init, b, u, max_iterations = 100, tol = 10e-6, mean = 0, std = 1):
    theta_es = theta_init
    for i in range(max_iterations):
        P = irt1pl(theta_es, b)
        Q = 1 - P
        L1 = 0
        L2 = 0
        for j in range(len(b)):
            L1 += (u[j] - P[j]) - (theta_es - mean) / std ** 2
            L2 += P[j] * Q[j] + 1 / std ** 2
        delta = L1 / L2
        if np.abs(delta) < tol:
            print(f"Hội tụ sau {i + 1} lần lặp !!")
            break
        theta_es = theta_es  + delta
    SE = 1 / np.sqrt(L2)
    return theta_es, SE

In [102]:
theta_es_1pl_map, se_1pl_map = uoc_luong_nlts_map_1pl(theta_init, b_true, responses1pl)

Hội tụ sau 3 lần lặp !!


In [103]:
print(theta_es_1pl_map, se_1pl_map )

0.22766130450160266 0.12794473801493056


In [104]:
def uoc_luong_nlts_map_2pl(theta_init, a, b, u, max_iterations = 100, tol = 10e-6, mean = 0, std = 1):
    theta_es = theta_init
    for i in range(max_iterations):
        P = irt2pl(theta_es, a, b)
        Q = 1 - P
        L1 = 0
        L2 = 0
        for j in range(len(b)):
            L1 += a[j] * (u[j] - P[j]) - (theta_es - mean) / std ** 2
            L2 += a[j] ** 2 * P[j] * Q[j] + 1 / std ** 2
        delta = L1 / L2
        if np.abs(delta) < tol:
            print(f"Hội tụ sau {i + 1} lần lặp !!")
            break
        theta_es = theta_es  + delta
    SE = 1 / np.sqrt(L2)
    return theta_es, SE

In [107]:
theta_es_2pl_map, se_2pl_map = uoc_luong_nlts_map_2pl(theta_init, a_true_2pl, b_true_2pl, responses1pl)

Hội tụ sau 3 lần lặp !!


In [108]:
print(theta_es_2pl_map, se_2pl_map )

0.16851727887588622 0.12552856239871046


In [117]:
def uoc_luong_nlts_map_3pl(theta_init, a, b, c, u, max_iterations = 100, tol = 10e-6, mean = 0, std = 1):
    theta_es = theta_init
    for i in range(max_iterations):
        P = irt3pl(theta_es, a, b, c)
        Q = 1 - P
        L1 = 0
        L2 = 0
        for j in range(len(b)):
            L1 += a[j] * (P[j] - c[j]) * (u[j] - P[j]) / ((1 - c[j]) * P[j]) - (theta_es - mean) / std ** 2
            L2 += a[j] ** 2 * (Q[j] * ((P[j] - c[j]) ** 2)) / (P[j] * ((1 - c[j]) ** 2)) + 1 / std ** 2
        delta = L1 / L2
        if np.abs(delta) < tol:
            print(f"Hội tụ sau {i + 1} lần lặp !!")
            break
        theta_es = theta_es  + delta
    SE = 1 / np.sqrt(L2)
    return theta_es, SE

In [118]:
theta_es_3pl_map, se_3pl_map = uoc_luong_nlts_map_3pl(theta_init, a_true_3pl, b_true_3pl, c_true_3pl, responses1pl)

Hội tụ sau 5 lần lặp !!


In [119]:
print(theta_es_3pl_map, se_3pl_map)

0.17766464118374864 0.13294390173663417
